In [4]:
# To ignore warning messages when filtering data
from warnings import filterwarnings
filterwarnings('ignore')

# References

1. https://techvidvan.com/tutorials/python-sentiment-analysis/
2. https://asperbrothers.com/blog/sentiment-analysis-in-python/
3. https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
4. https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

# Basic dataset preparation

## Columns of the full dataset

In [5]:
import pandas as pd
# The whole data set
data = pd.read_csv("../data/amazonConsumerReviews.csv")
print("COLUMN NAMES\n------------")
for c in data.columns: print(c)

COLUMN NAMES
------------
id
dateAdded
dateUpdated
name
brand
categories
primaryCategories
manufacturer
manufacturerNumber
reviews.date
reviews.doRecommend
reviews.numHelpful
reviews.rating
reviews.text
reviews.title


## Only keeping relevant columns

In [6]:
# Only selecting relevant columns
reviewsData = data[['id',
                  'reviews.doRecommend',
                  'reviews.rating',
                  'reviews.text',
                  'reviews.title']]
reviewsData.head(3)

,id,reviews.doRecommend,reviews.rating,reviews.text,reviews.title
0,AVqVGZNvQMlgsOJE6eUY,False,3,I thought it would be as big as small paper bu...,Too small
1,AVqVGZNvQMlgsOJE6eUY,True,5,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach
2,AVqVGZNvQMlgsOJE6eUY,True,4,Didnt know how much i'd use a kindle so went f...,Great for the price


## Converting ratings into sentiment labels

In [7]:
print("REVIEWS RATING INFO\n------------")
ratings = reviewsData['reviews.rating']
print("Minimum:", min(ratings))
print("Maximum:", max(ratings))
print("Mean:", sum(ratings)/len(ratings))

REVIEWS RATING INFO
------------
Minimum: 1
Maximum: 5
Mean: 4.5968


For our purposes, let rating < 3 mean negative, rating > 3 mean positive, and rating = 3 be neutral.

In [8]:
# Converting ratings to sentiment labels
sentiment = []
for r in ratings:
    if r < 3: sentiment.append(0)   # Negative
    elif r > 3: sentiment.append(1) # Positive
    else: sentiment.append('n')     # Neutral
reviewsData['sentiment'] = sentiment
try: del(reviewsData['reviews.rating'])
except: pass
reviewsData.head(3)

,id,reviews.doRecommend,reviews.text,reviews.title,sentiment
0,AVqVGZNvQMlgsOJE6eUY,False,I thought it would be as big as small paper bu...,Too small,n
1,AVqVGZNvQMlgsOJE6eUY,True,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,1
2,AVqVGZNvQMlgsOJE6eUY,True,Didnt know how much i'd use a kindle so went f...,Great for the price,1


## Removing neutral sentiment rows

In [9]:
# Removing neutral rows
reviewsData = reviewsData[reviewsData['sentiment'] != 'n']
# Hence we can see the first row (which has neutral sentiment label) will be removed
reviewsData.head(3)

,id,reviews.doRecommend,reviews.text,reviews.title,sentiment
1,AVqVGZNvQMlgsOJE6eUY,True,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,1
2,AVqVGZNvQMlgsOJE6eUY,True,Didnt know how much i'd use a kindle so went f...,Great for the price,1
3,AVqVGZNvQMlgsOJE6eUY,True,I am 100 happy with my purchase. I caught it o...,A Great Buy,1


# Text processing and tokenization

The processes are elaborated and explained in the notebook on text mining and tokenization.

## Tokenization & lemmatization

In [58]:
# Instantiating tokenizer object
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 5000)
    
# Creating internal vocabulary and word indices
reviews = reviewsData['reviews.text'].values
tokenizer.fit_on_texts(reviews)

## Encoding the the tokens

In [60]:
# Replacing words with their respective indices
# (Indices can be seen in the 'word_index' or 'index_word'attributes)
encodedDocs = tokenizer.texts_to_sequences(reviews)
# NOTE: reviews = reviewsData['reviews.text'].values

# Machine learning approach

## Need for machine learning & deep learning

Natural language is extremely diverse in terms of concepts and expressions, and varies greatly depending on context. Furthermore, the number of permutations and combinations in which concepts and expressions may appear are practically infinite, and the effects of these permutations and combinations on meaning are often significant.
<br><br>
However, language is based on rules, which may be clearly defined or implicit in usage. Usage of language usually follows discernible patterns that human speakers learn to identify over time, through experience and education.
<br><br>
Due to the necessity to handle unstructured data (i.e. natural language texts), and due to the complexity of natural language that makes it difficult to structure it meaningfully using hard computing instructions, we will use **deep learning**, which uses artificial neural networks with multiple layers.

**Deep learning**: Deep learning is a subfield of machine learning. It refers to the usage of an ANN using more than one layer, hence the term 'deep', which refers to depth of network layers. Multiple network layers enhances the ANN's ability to adapt to more diverse and complex problems, since there are many more nodes, hence many more link weights and link structures to modify according to requirements.

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
#------------------------
vocabularySize = len(encodedDocs)
embeddingVectorLength = 32
#------------------------
model = Sequential()
model.add(Embedding(vocabularySize,
                    embeddingVectorLength,
                    input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50,
          dropout=0.5,
          recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 32)           153696    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 200, 32)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 50)                16600     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 170,347
Trainable params: 170,347
Non-trainable params: 0
________________________________________________

**_Embedding_**

The class 'Embedding' inherits from the classes 'Layer' and 'Module'. The arguments used above are
1. **input_dim (int ≥ 1)**: size of vocabulary
2. **output_dim (int ≥ 1)**: dimension of dense embedding
3. **input_lenght**: Length of input sequences, when it is constant. This argument is required if you are going to connect Flatten then Dense layers upstream (without it, the shape of the dense outputs cannot be computed).

In [77]:
import numpy as np
def intstr(x): return str(int(x))
reviewsData['sentiment'] = list(map(intstr, reviewsData['sentiment']))
sentimentLabels = np.array(reviewsData['sentiment'])
from tensorflow.keras.preprocessing.sequence import pad_sequences
paddedSequence = pad_sequences(encodedDocs, maxlen=200)

In [79]:
history = model.fit(paddedSequence,
                    sentimentLabels,
                    validation_split=0.2,
                    epochs=2,
                    batch_size=32)

Epoch 1/2


UnimplementedError:  Cast string to float is not supported
	 [[node binary_crossentropy/Cast
 (defined at /Users/pranav/opt/anaconda3/lib/python3.8/site-packages/keras/losses.py:1797)
]] [Op:__inference_train_function_4466]

Errors may have originated from an input operation.
Input Source operations connected to node binary_crossentropy/Cast:
In[0] ExpandDims (defined at /Users/pranav/opt/anaconda3/lib/python3.8/site-packages/keras/engine/compile_utils.py:677)

Operation defined at: (most recent call last)
>>>   File "/Users/pranav/opt/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/Users/pranav/opt/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/Users/pranav/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/Users/pranav/opt/anaconda3/lib/python3.8/site

In [78]:
paddedSequence

array([[   0,    0,    0, ...,   58,    1, 1633],
       [   0,    0,    0, ...,    7,  116,  623],
       [   0,    0,    0, ...,   47,    9,  295],
       ...,
       [   0,    0,    0, ...,    7,   92, 1471],
       [   0,    0,    0, ...,    7,  197,   85],
       [   0,    0,    0, ...,   38,   50, 2083]], dtype=int32)